# **Data Exploration**

In [ ]:
# Importing Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sheets=pd.read_excel('/content/Regional Sales Dataset.xlsx', sheet_name=None)

In [ ]:
sheets

In [ ]:
# Assigning dataframes to each sheet

df_sales=sheets['Sales Orders']
df_customers=sheets['Customers']
df_products=sheets['Products']
df_regions=sheets['Regions']
df_state_reg=sheets['State Regions']
df_budget=sheets['2017 Budgets']

In [ ]:
dfs = {
    "Sales": df_sales,
    "Customers": df_customers,
    "Products": df_products,
    "Regions": df_regions,
    "State_Region": df_state_reg,
    "Budget": df_budget
}

for name, df2 in dfs.items():
    print(f"\n Overview of {name}:\n")
    print(df2.head(5))
    print(f"\nShape of {name}: {df2.shape}\n{'-'*60}")

In [ ]:
# Adjusting Header

header=df_state_reg.iloc[0]
df_state_reg.columns=header
df_state_reg=df_state_reg[1:].reset_index(drop=True)

df_state_reg.head(5)

In [ ]:
# Checking for null values

for name, df2 in dfs.items():
    print(f"\n Null values in {name}:\n")
    print(f"{df2.isnull().sum()}\n{'-'*60}")

In [ ]:
# Merging Sheets

merge_plan = [
    (df_customers, "Customer Name Index", "Customer Index"),
    (df_products, "Product Description Index", "Index"),
    (df_regions, "Delivery Region Index", "id")
]

df = df_sales.copy()

for df2, left_col, right_col in merge_plan:
    df = df.merge(
        df2,
        how="left",
        left_on=left_col,
        right_on=right_col
    )

In [ ]:
print(df.columns.values)

In [ ]:
df.info()

In [ ]:
# Merging the df_state_reg for selected columns
df=df.merge(
    df_state_reg[['State Code','Region']],
    how='left',
    left_on='state_code',
    right_on='State Code'
)

In [ ]:
# Merging the budget data in dataframe
df=df.merge(
    df_budget,
    how='left',
    left_on='Product Name',
    right_on='Product Name'
)

In [ ]:
print(df.columns.values)

In [ ]:
df.head(5)

In [ ]:
df.to_csv('file.csv')

In [ ]:
# Dropping Duplicate Columns

drop_cols=['Customer Index','Index','id','State Code']
df=df.drop(columns=drop_cols)

print(df.head(5))

In [ ]:
# Converting column names to lowercase for consistency and easy accessibility

df.columns = df.columns.str.lower()

In [ ]:
print(df.columns.values)

In [ ]:
print(df.head(5))

In [ ]:
print(df.columns.values)

In [ ]:
# Finalising the dataframe by keeping the necessary columns

keep_cols=['ordernumber','orderdate','customer names','channel','product name','order quantity','unit price',
           'line total','total unit cost','state_code','county','state','region','latitude','longitude','2017 budgets']

df=df[keep_cols]

In [ ]:
df.head(5)

In [ ]:
df.columns.values

In [ ]:
# Renaming the columns
df = df.rename(columns={
                    'ordernumber': 'order_number',
                    'orderdate': 'order_date',
                    'customer names': 'customer_name',
                    'product name': 'product_name',
                    'order quantity': 'order_quantity',
                    'unit price': 'unit_price',
                    'line total': 'revenue',
                    'total unit cost': 'cost',
                    'state_code': 'state',
                    'state': 'state_name',
                    'latitude': 'lat',
                    'longitude': 'lon',
                    '2017 budgets': 'budget'
})

In [ ]:
df.head(1)

In [ ]:
# Providing Nan for the budget which is not in the year 2017

df.loc[df['order_date'].dt.year != 2017, 'budget'] = pd.NA

df[['order_date','product_name','revenue','budget']].head(5)

# **Feature Engineering**

In [ ]:
df['cogs']=df['order_quantity']*df['cost']
df['profit']=df['revenue']-df['cogs']
df['profit_margin']=df['profit']/df['revenue']*100
df.head(5)

# **EDA**

In [ ]:
# Plotting Monthly Rental Trend

df['order_month']=df['order_date'].dt.to_period('M')
monthly_sales=df.groupby('order_month')['revenue'].sum()
plt.figure(figsize=(15,4))
monthly_sales.plot(marker='o', color='navy')
from matplotlib.ticker import FuncFormatter
formatter=FuncFormatter(lambda x, pos:f'{x/1e6:1f}M')
plt.gca().yaxis.set_major_formatter(formatter)
plt.title('Monthly Sale Trend')
plt.xlabel('Month')
plt.ylabel('Revenue (In Millions)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Ensure datetime
df["order_date"] = pd.to_datetime(df["order_date"])

# Create df_new without 2018 data
df_new = df[df["order_date"].dt.year != 2018].copy()

# -------------------------------
# 1. Monthly Sales Trend (time series across years)
# -------------------------------
df_new["Month"] = df_new["order_date"].dt.to_period("M")
monthly_sales = df_new.groupby("Month")["revenue"].sum().reset_index()
monthly_sales["Month"] = monthly_sales["Month"].dt.to_timestamp()

# Calculate percentiles
p25 = monthly_sales["revenue"].quantile(0.25)
p50 = monthly_sales["revenue"].quantile(0.50)
p75 = monthly_sales["revenue"].quantile(0.75)

plt.figure(figsize=(12,6))
plt.plot(monthly_sales["Month"], monthly_sales["revenue"], marker="o", linewidth=2, color="teal")

# Add percentile lines
plt.axhline(p25, color="red", linestyle="--", linewidth=1, label="25th Percentile")
plt.axhline(p50, color="orange", linestyle="--", linewidth=1, label="50th Percentile (Median)")
plt.axhline(p75, color="green", linestyle="--", linewidth=1, label="75th Percentile")

plt.title("Monthly Sales Trend", fontsize=16)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Revenue", fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Aggregated Monthly Sales across years

# Ensure datetime
df["order_date"] = pd.to_datetime(df["order_date"])

# Create df_new without 2018 data
df_new = df[df["order_date"].dt.year != 2018].copy()

df_new['MonthNum']=df_new['order_date'].dt.month
df_new['MonthName']=df_new['order_date'].dt.strftime('%b')

monthly_sales_seasonal = df_new.groupby(["MonthNum", "MonthName"])["revenue"].sum().reset_index()
monthly_sales_seasonal = monthly_sales_seasonal.sort_values("MonthNum")

# Calculate percentiles
p25_seasonal = monthly_sales_seasonal["revenue"].quantile(0.25)
p50_seasonal = monthly_sales_seasonal["revenue"].quantile(0.50)
p75_seasonal = monthly_sales_seasonal["revenue"].quantile(0.75)

plt.figure(figsize=(10,6))
plt.plot(monthly_sales_seasonal["MonthName"], monthly_sales_seasonal["revenue"],
         marker="o", linewidth=2, color="teal")

# Add percentile lines
plt.axhline(p25_seasonal, color="red", linestyle="--", linewidth=1, label="25th Percentile")
plt.axhline(p50_seasonal, color="orange", linestyle="--", linewidth=1, label="50th Percentile (Median)")
plt.axhline(p75_seasonal, color="green", linestyle="--", linewidth=1, label="75th Percentile")

plt.title("Overall Monthly Sales Trend", fontsize=16)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Revenue", fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# PLotting YoY Comparison trend across years and months

# Extract Year and Month
df_new["Year"] = df_new["order_date"].dt.year
df_new["MonthNum"] = df_new["order_date"].dt.month
df_new["MonthName"] = df_new["order_date"].dt.strftime("%b")

# Group by Year-Month
monthly_sales = (
    df_new.groupby(["Year", "MonthNum", "MonthName"])["revenue"]
    .sum()
    .reset_index()
)

# Pivot for YoY comparison
monthly_pivot = monthly_sales.pivot(index="MonthNum", columns="Year", values="revenue")

# Add month names for x-axis
monthly_pivot["MonthName"] = monthly_pivot.index.map(
    lambda x: pd.to_datetime(str(x), format="%m").strftime("%b")
)

# Plot
plt.figure(figsize=(12,6))
for year in monthly_pivot.columns[:-1]:  # exclude "MonthName"
    plt.plot(
        monthly_pivot["MonthName"],
        monthly_pivot[year],
        marker="o",
        linewidth=2,
        label=str(year)
    )

plt.title("Monthly Sales Trend (YoY Comparison)", fontsize=16)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Revenue", fontsize=12)
plt.legend(title="Year")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
df_new.head(5)

In [ ]:
# Top 10 products by revenue
top_products = (
    df_new.groupby("product_name")["revenue"]
    .sum()
    .reset_index()
    .sort_values(by="revenue", ascending=False)
    .head(10)
)

# Colors: green for top, red for bottom, teal for others
colors = ["green" if i == 0 else "red" if i == len(top_products)-1 else "teal"
          for i in range(len(top_products))]

plt.figure(figsize=(10,6))
bars = plt.barh(top_products["product_name"], top_products["revenue"], color=colors)
plt.gca().invert_yaxis()  # Highest at top

# Add labels to bars
for bar in bars:
    width = bar.get_width()
    plt.text(width + (0.01 * top_products["revenue"].max()),  # position to the right
             bar.get_y() + bar.get_height()/2,
             f"{width:,.0f}",
             va='center', fontsize=10)

# Titles and labels
plt.title("Top 10 Products by Revenue", fontsize=16)
plt.xlabel("Revenue", fontsize=12)
plt.ylabel("Product", fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Sales Distribution by Channel
channel_sales = (
    df_new.groupby("channel")["revenue"]
    .sum()
    .reset_index()
    .sort_values(by="revenue", ascending=False)
)

# Plot pie chart
plt.figure(figsize=(6,6))
plt.pie(channel_sales["revenue"],
        labels=channel_sales["channel"],
        autopct='%1.1f%%',
        startangle=140,
        colors=plt.cm.Set3.colors)

plt.title("Sales Distribution by Channel", fontsize=16)
plt.show()

In [ ]:
# Average Order Value Distribution

aov=df_new.groupby('order_number')['revenue'].sum()

plt.figure(figsize=(10,6))
plt.hist(aov,bins=50,color='teal',edgecolor='black')
plt.title('Average Order Value Distribution')
plt.xlabel('AOV')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Unit Price Distribution Per Product (By Median)

# Compute median (or mean) unit price to use for sorting
product_order = (
    df_new.groupby("product_name")["unit_price"]
    .median()
    .sort_values(ascending=False)
    .index
)

# Plot boxplot sorted by unit price
plt.figure(figsize=(12,6))
sns.boxplot(
    data=df_new,
    x="product_name",
    y="unit_price",
    order=product_order,
    palette="Set2"
)

plt.xticks(rotation=45, ha="right")
plt.title("Unit Price Distribution per Product (Sorted by Median Unit Price)", fontsize=16)
plt.xlabel("Product", fontsize=12)
plt.ylabel("Unit Price", fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Top 10 States By Revenue And Order Count

state_stats = (
    df.groupby("state_name")
    .agg(
        revenue=("revenue", "sum"),
        order_count=("order_number", "nunique")  # unique orders
    )
    .reset_index()
)

# --- Get top 10 states by revenue ---
top_states = state_stats.nlargest(10, "revenue")

# --- Plotting side-by-side bar charts ---
fig, axes = plt.subplots(1, 2, figsize=(16,6))

# Revenue plot
sns.barplot(
    data=top_states.sort_values("revenue", ascending=False),
    x="revenue", y="state_name", ax=axes[0], palette="Blues_r"
)
axes[0].set_title("Top 10 States by Revenue", fontsize=14)
axes[0].set_xlabel("Revenue")
axes[0].set_ylabel("State")

# Order count plot
sns.barplot(
    data=top_states.sort_values("order_count", ascending=False),
    x="order_count", y="state_name", ax=axes[1], palette="Greens_r"
)
axes[1].set_title("Top 10 States by Order Count", fontsize=14)
axes[1].set_xlabel("Order Count")
axes[1].set_ylabel("")

plt.tight_layout()
plt.show()

In [ ]:
# Average Profit Margin By Channel

plt.figure(figsize=(6,4))
ax = sns.barplot(
    data=df,
    x="channel",
    y="profit_margin",
    estimator="mean",
    palette="Reds_r"
)

# Add labels (no extra % scaling)
for p in ax.patches:
    value = p.get_height()
    ax.annotate(f"{value:.2f}%",   # just add % sign
                (p.get_x() + p.get_width() / 2., value),
                ha='center', va='bottom',
                fontsize=9, color='black', xytext=(0,3),
                textcoords='offset points')

ax.set_title("Average Profit Margin by Channel", fontsize=12)
ax.set_xlabel("Channel", fontsize=10)
ax.set_ylabel("Avg Profit Margin (%)", fontsize=10)
plt.xticks(rotation=30, ha="right", fontsize=9)
plt.yticks(fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
# Customer Segmentation: Revenue vs Profit Margin

customer_stats = (
    df.groupby("customer_name")
    .agg(
        revenue=("revenue", "sum"),
        profit_margin=("profit_margin", "mean")
    )
    .reset_index()
)

plt.figure(figsize=(8,6))
sns.scatterplot(
    data=customer_stats,
    x="revenue",
    y="profit_margin",
    hue="revenue",  # optional: color by revenue
    palette="viridis",
    size="revenue",
    sizes=(20, 200),
    alpha=0.7,
    legend=False
)
plt.title("Customer Segmentation: Revenue vs Profit Margin", fontsize=14)
plt.xlabel("Revenue")
plt.ylabel("Profit Margin")
plt.tight_layout()
plt.show()

In [ ]:
# Correlation Heatmap

plt.figure(figsize=(10,6))
corr = df[["order_quantity","unit_price","revenue","cogs","profit","profit_margin"]].corr()

sns.heatmap(
    corr,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=0
)
plt.title("Correlation Heatmap", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
df.head(5)

In [ ]:
df.to_csv('final_df.csv')

In [ ]:
# Defining the fact and dimension tables for data model

import os
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype as is_datetime

# ---------- Helpers
def ensure_dir(path):
    d = os.path.dirname(path)
    if d and not os.path.exists(d):
        os.makedirs(d, exist_ok=True)

def make_dim_from_series(series: pd.Series, key_col: str, val_col: str) -> pd.DataFrame:
    """Create a dimension with Unknown=0 and a stable surrogate key using factorize."""
    s = series.astype("string")
    uniques = pd.Series(pd.unique(s.dropna())).sort_values(ignore_index=True)
    dim_rest = pd.DataFrame({val_col: uniques})
    dim_rest.insert(0, key_col, range(1, len(dim_rest) + 1))
    # Unknown row
    dim_unknown = pd.DataFrame({key_col: [0], val_col: ["Unknown"]})
    dim = pd.concat([dim_unknown, dim_rest], ignore_index=True)
    return dim

def map_key(series: pd.Series, dim: pd.DataFrame, key_col: str, val_col: str) -> pd.Series:
    """Map natural value to surrogate key; Unknown→0; return Int64."""
    m = dict(zip(dim[val_col], dim[key_col]))
    return series.astype("string").map(m).fillna(0).astype("Int64")

# ---------- Prep
df = df.copy()

# Make sure order_date is datetime
if not is_datetime(df["order_date"]):
    df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce")

# Base fact columns (keep order_date)
fact_sales = df[[
    "order_number", "order_date", "customer_name", "product_name", "state",
    "order_quantity", "unit_price", "revenue", "cost", "cogs", "profit",
    "profit_margin", "budget"
]].copy()

# ---------- Dim Customer
dim_customer = make_dim_from_series(df["customer_name"], "CustomerID", "CustomerName")
fact_sales["CustomerID"] = map_key(fact_sales["customer_name"], dim_customer, "CustomerID", "CustomerName")
fact_sales.drop(columns=["customer_name"], inplace=True)

# ---------- Dim Product
dim_product = make_dim_from_series(df["product_name"], "ProductID", "ProductName")
fact_sales["ProductID"] = map_key(fact_sales["product_name"], dim_product, "ProductID", "ProductName")
fact_sales.drop(columns=["product_name"], inplace=True)

# ---------- Dim Geography
geo_cols = ["state", "state_name", "county", "region", "lat", "lon"]
missing_geo = [c for c in geo_cols if c not in df.columns]
for c in missing_geo:
    df[c] = pd.NA

geo_natural = df[geo_cols].astype({
    "state":"string","state_name":"string","county":"string","region":"string"
}).copy()

dim_geo_unique = geo_natural.drop_duplicates(ignore_index=True)
dim_geo_unique.insert(0, "GeoID", range(1, len(dim_geo_unique) + 1))

dim_geo_unknown = pd.DataFrame(
    {"GeoID":[0], "state":["Unknown"], "state_name":["Unknown"], "county":["Unknown"],
     "region":["Unknown"], "lat":[pd.NA], "lon":[pd.NA]}
)
dim_geo = pd.concat([dim_geo_unknown, dim_geo_unique], ignore_index=True)

# Map state → GeoID
state_to_geoid = (
    dim_geo.dropna(subset=["state"])
           .drop_duplicates(subset=["state"])[["state","GeoID"]]
           .set_index("state")["GeoID"]
           .to_dict()
)
fact_sales["GeoID"] = fact_sales["state"].astype("string").map(state_to_geoid).fillna(0).astype("Int64")
fact_sales.drop(columns=["state"], inplace=True)

# ---------- Clean types
fact_sales["CustomerID"] = fact_sales["CustomerID"].astype("Int64")
fact_sales["ProductID"]  = fact_sales["ProductID"].astype("Int64")
fact_sales["GeoID"]      = fact_sales["GeoID"].astype("Int64")

dim_customer["CustomerID"] = dim_customer["CustomerID"].astype("Int64")
dim_product["ProductID"]   = dim_product["ProductID"].astype("Int64")
dim_geo["GeoID"]           = dim_geo["GeoID"].astype("Int64")

# ---------- Save to CSV
outdir = "./exports"  # change if you like
ensure_dir(f"{outdir}/dummy.txt")

fact_sales.to_csv(f"{outdir}/FactSales.csv", index=False)
dim_customer.to_csv(f"{outdir}/DimCustomer.csv", index=False)
dim_product.to_csv(f"{outdir}/DimProduct.csv", index=False)
dim_geo.to_csv(f"{outdir}/DimGeography.csv", index=False)

# ---------- Validations
print("Fact rows:", len(fact_sales))
print("DimCustomer rows:", len(dim_customer), "unique CustomerID?", dim_customer["CustomerID"].is_unique)
print("DimProduct rows:", len(dim_product), "unique ProductID?", dim_product["ProductID"].is_unique)
print("DimGeography rows:", len(dim_geo), "unique GeoID?", dim_geo["GeoID"].is_unique)
print("Any null keys in fact?",
      fact_sales[["CustomerID","ProductID","GeoID"]].isna().any().to_dict())